In [1]:
push!(LOAD_PATH, "../src")
using Quaternions
using FixedSizeArrays
using RigidBodyDynamics

In [2]:
mechanism = Mechanism{Float64}("world")
world = root_body(mechanism)

RigidBody: "world"

In [3]:
body1 = RigidBody(rand(SpatialInertia{Float64}, CartesianFrame3D("body1")));
joint1 = Joint("1", QuaternionFloating());
joint1ToWorld = rand(Transform3D{Float64}, joint1.frameBefore, world.frame)
body1ToJoint1 = rand(Transform3D{Float64}, body1.frame, joint1.frameAfter)
attach!(mechanism, world, joint1, joint1ToWorld, body1, body1ToJoint1);

In [4]:
joint2 = Joint("2", Revolute(rand(Vec{3, Float64})));
joint2ToBody1 = rand(Transform3D{Float64}, joint2.frameBefore, body1.frame)
body2frame = CartesianFrame3D("body2")
body2 = RigidBody(rand(SpatialInertia{Float64}, body2frame));
body2ToJoint2 = rand(Transform3D{Float64}, body2frame, joint2.frameAfter)
attach!(mechanism, body1, joint2, joint2ToBody1, body2, body2ToJoint2);

In [5]:
mechanism

Vertex: world (root)
  Vertex: body1, Edge: 1
    Vertex: body2, Edge: 2

In [6]:
x = MechanismState{Float64}(mechanism);

In [7]:
rand!(x)
x.v

DataStructures.OrderedDict{RigidBodyDynamics.Joint,Array{Float64,1}} with 2 entries:
  Joint "1": Quaternion f… => [0.4622783520757443,0.35095641849642467,0.3979410…
  Joint "2": Revolute joi… => [0.1091996633351211]

In [8]:
num_positions(x)

8

In [9]:
num_velocities(x)

7

In [10]:
cache = MechanismStateCache(mechanism, x);

In [11]:
relative_transform(cache, joint2.frameAfter, body1.frame)

Transform3D from "after_2" to "body1":
rotation: 1.8022680547849568 rad about [-0.6261660807166802,0.7569762845378246,-0.18682329621121246], translation: Vec(0.06041630208937121,0.4121887769843865,0.549931407476038)


In [12]:
center_of_mass(cache)

Point3D in "world": Vec(0.23422452021113332,-0.9330895087013807,-0.012199150848194949)

In [13]:
mass(mechanism)

0.3169488774459739

In [14]:
transform(cache, Point3D{Float64}(joint2.frameAfter), root_frame(mechanism))

Point3D in "world": Vec(0.5861844517597731,-1.2264928871804217,0.639372620417482)

In [15]:
relative_twist(cache, body2, body1)

Twist of "after_2" w.r.t "after_1" in "world":
angular: Vec(-0.024024947249948303,0.053636136373954674,-0.08109522370409566), linear: Vec(0.06516923799489366,0.03217586576873023,0.0019742422779325253)

In [16]:
relative_twist(cache, body1, world) + relative_twist(cache, body2, body1) + relative_twist(cache, world, body2)

Twist of "world" w.r.t "world" in "world":
angular: Vec(0.0,0.0,0.0), linear: Vec(0.0,0.0,0.0)

In [17]:
geometric_jacobian(cache, path(mechanism, body2, world))

GeometricJacobian: body: "world", base: "after_2", expressed in "world":
6x7 Array{Float64,2}:
  0.220009    0.577714    0.121913  …  -0.0        -0.0       -0.0     
 -0.491175   -0.811987    0.186632     -0.0        -0.0       -0.0     
  0.742633    0.0832059   0.974836     -0.0        -0.0       -0.0     
 -0.59679     0.239881   -0.552669      0.577714    0.121913   0.807083
 -0.294652    0.115448   -0.951891     -0.811987    0.186632   0.553033
 -0.0180792  -0.538905    0.251356  …   0.0832059   0.974836  -0.206812

In [18]:
mass_matrix(cache)

7x7 Array{Float64,2}:
  1.21283     0.0193577    -0.858681     …   0.166837      0.587689 
  0.0193577   0.470599      0.69064         -0.041997      0.0210213
 -0.858681    0.69064       3.18548          0.0          -0.641422 
  0.0         0.253249     -0.166837         3.1225e-17   -0.0385941
 -0.253249   -6.93889e-18   0.041997         9.71445e-17   0.0417142
  0.166837   -0.041997      8.67362e-19  …   0.316949      0.0194269
  0.587689    0.0210213    -0.641422         0.0194269     0.550441 

In [19]:
A = momentum_matrix(cache)

MomentumMatrix expressed in "world":
6x7 Array{Float64,2}:
 -0.135247  -0.690414   -2.09584    …   0.287578   -0.0633011   0.209628  
  1.13313   -0.462685   -2.52857        0.0728405  -0.0122325   0.86263   
 -0.395211  -0.140406   -0.0434839     -0.0499097  -0.279744   -0.252134  
 -0.103777  -0.11241     0.0912641     -0.0386401  -0.255804    0.00153169
 -0.045002   0.22886    -0.143307      -0.0591528  -0.175283   -0.0498668 
  0.28138   -0.0297573  -0.0270584  …  -0.308973    0.0655487  -0.0334355 

In [20]:
A * velocity_vector(x)

Momentum expressed in "world":
angular: Vec(-0.9675748577497181,-0.5125026508832913,-0.3406167600411281), linear: Vec(-0.11745479389735165,-0.01039360131617528,-0.05390770174332112)

In [21]:
accel = SpatialAcceleration(joint2.frameAfter, world.frame, joint2.frameAfter, rand(Vec{3, Float64}), rand(Vec{3, Float64}))
transform(cache, accel, world.frame)

SpatialAcceleration of "after_2" w.r.t "world" in "world":
angular: Vec(-0.18086615825264782,0.15852549700145835,-0.7847600665341297), linear: Vec(0.35958664142329255,0.374917868724429,-0.8688854441688401)

In [22]:
bias_acceleration(cache, body2)

SpatialAcceleration of "after_2" w.r.t "after_1" in "world":
angular: Vec(0.008717627775600367,-0.04400649338280775,-0.03168841160423989), linear: Vec(0.043708626142588775,-0.008178289640613522,-0.029584180831111598)

In [23]:
v̇ = [j::Joint => rand(num_velocities(j))::Vector{Float64} for j in joints(mechanism)]
τ = inverse_dynamics(cache, v̇)

7-element Array{Float64,1}:
 -2.50813 
  0.619299
  2.33788 
  0.301742
  3.22355 
 -0.524155
  0.391491

In [24]:
rand_chain_mechanism(Float64, QuaternionFloating, Revolute{Float64}, Revolute{Float64})

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body2, Edge: joint2
      Vertex: body3, Edge: joint3

In [25]:
rand_tree_mechanism(Float64, [QuaternionFloating; [Revolute{Float64} for i = 1 : 10]]...)

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body2, Edge: joint2
      Vertex: body7, Edge: joint7
    Vertex: body9, Edge: joint9
  Vertex: body3, Edge: joint3
    Vertex: body5, Edge: joint5
      Vertex: body6, Edge: joint6
      Vertex: body8, Edge: joint8
    Vertex: body10, Edge: joint10
  Vertex: body4, Edge: joint4
    Vertex: body11, Edge: joint11